<a href="https://colab.research.google.com/github/SeongGwan-a/Project-Classification-of-novelists/blob/main/RNN/glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 모듈

from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 데이터 로드

train = pd.read_csv("/content/drive/MyDrive/소설작가분류/train.csv")
test = pd.read_csv("/content/drive/MyDrive/소설작가분류/test_x.csv")

In [ ]:
# 결측값 확인

train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54879 entries, 0 to 54878
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   54879 non-null  int64 
 1   text    54879 non-null  object
 2   author  54879 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19617 entries, 0 to 19616
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   19617 non-null  int64 
 1   text    19617 non-null  object
dtypes: int64(1), object(1)
memory usage: 306.6+ KB


(None, None)

In [ ]:
train.replace("", float("NaN"), inplace=True)

In [ ]:
# 결측값 확인

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54879 entries, 0 to 54878
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   54879 non-null  int64 
 1   text    54879 non-null  object
 2   author  54879 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
# 중복값 확인

train['text'].nunique(), test['text'].nunique()

(54744, 19606)

In [ ]:
# train data에서 135개 중복값 제거 (54879 - 54744 = 135)

train_clean = train.drop_duplicates(subset=['text'])

In [ ]:
# 부호와 불용어 제거 함수

'''def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)


stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopword2 = stopwords.words('english')

In [ ]:
# 소문자 변환

train_clean['text'] = train_clean['text'].str.lower()
test['text'] = test['text'].str.lower()

In [ ]:
X_data = train_clean['text'].values
X_test = test['text'].values
y_data = train_clean['author'].values

In [ ]:
X_data.shape

(54744,)

In [ ]:
vocab_size = 42330

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_data)
word_index = tokenizer.word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_data)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
len(train_sequences)

54744

In [ ]:
max_len=max(len(l) for l in train_sequences)
print(max_len)

473


In [ ]:
trn = pad_sequences(train_sequences, maxlen = max_len, padding='post')
tst = pad_sequences(test_sequences, maxlen=max_len, padding='post')
print(trn.shape, tst.shape)

(54744, 473) (19617, 473)


In [ ]:
glove = open('/content/drive/MyDrive/소설작가분류/glove.6B.100d.txt', encoding="utf8")
w2v = open('/content/drive/MyDrive/소설작가분류/GoogleNews-vectors-negative300.bin', encoding="utf8")

In [ ]:
glove_dict = dict()

glove = open('/content/drive/MyDrive/소설작가분류/glove.6B.100d.txt', encoding="utf8")

for line in glove:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    glove_dict[word] = word_vector_arr
glove.close()
print('%s개의 Embedding vector가 있습니다.' % len(glove_dict))

400000개의 Embedding vector가 있습니다.


In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.

In [ ]:
for word, i in word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = glove_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임시 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

In [ ]:
'''model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

In [ ]:
target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [ ]:
'''from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
def get_model():
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64)),
        Dense(n_class, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    return model

In [ ]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn, y_data), 1):
    print(f'training model for CV #{i}')
    clf = get_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf.fit(trn[i_trn], 
            to_categorical(y_data[i_trn]),
            validation_data=(trn[i_val], to_categorical(y_data[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

training model for CV #1
Epoch 1/10
86/86 [==============================] - 791s 9s/step - loss: 1.2614 - val_loss: 1.0421
Epoch 2/10
86/86 [==============================] - 779s 9s/step - loss: 0.9363 - val_loss: 0.8920
Epoch 3/10
86/86 [==============================] - 776s 9s/step - loss: 0.7847 - val_loss: 0.8006
Epoch 4/10
86/86 [==============================] - 778s 9s/step - loss: 0.6769 - val_loss: 0.7408
Epoch 5/10
86/86 [==============================] - 782s 9s/step - loss: 0.5930 - val_loss: 0.7402
Epoch 6/10
86/86 [==============================] - 777s 9s/step - loss: 0.5226 - val_loss: 0.7253
Epoch 7/10
86/86 [==============================] - 780s 9s/step - loss: 0.4459 - val_loss: 0.7615
Epoch 8/10
86/86 [==============================] - 777s 9s/step - loss: 0.3904 - val_loss: 0.8025
Epoch 9/10
86/86 [==============================] - 781s 9s/step - loss: 0.3422 - val_loss: 0.8322
Epoch 00009: early stopping
training model for CV #2
Epoch 1/10
86/86 [=============

In [ ]:
print(clf.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 473, 100)          4233000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 473, 128)          84480     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 4,416,941
Trainable params: 183,941
Non-trainable params: 4,233,000
_________________________________________________________________
None


In [ ]:
glove = pd.read_csv("/content/drive/MyDrive/소설작가분류/sample_submission.csv", index_col=0)
print(glove.shape)
glove.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [ ]:
glove[glove.columns] = p_tst
glove.head()

,0,1,2,3,4
index,,,,,
0,0.030622,0.512434,0.137687,0.318001,0.001257
1,0.095761,0.532137,0.021741,0.334997,0.015363
2,0.934131,0.009992,0.009206,0.008183,0.038488
3,0.000939,0.014082,0.981871,0.000552,0.002556
4,0.881813,0.035733,0.018257,0.027389,0.036808


In [ ]:
glove.to_csv("glove.csv")